In [3]:
!pip install pylidc
!pip install pydicom
!pip install opencv-python

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.3 MB/s eta 0:00:0000:0100:01


In [4]:
import numpy as np
import pylidc as pl
import pydicom
import cv2
from statistics import median_high
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, AveragePooling2D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score


In [6]:
import os
import pydicom
import pylidc as pl
import numpy as np
import cv2
from pylidc.Contour import Contour
from statistics import median_high

# Paths to the directories containing DICOM files
base_dirs = [
    "/kaggle/input/lidcidri30/LIDC-IDRI-0001-0200",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0201-0400",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0401-0600"
]

images = []
labels = []
malignant_count = 0
benign_count = 0
max_malignant_count = 4500  # Target number of malignant images
max_benign_count = 2500     # Target number of benign images

# Patch the Contour class method to_matrix
def patched_to_matrix(self, include_k=True):
    coords = np.array([list(map(int, coord.split(','))) for coord in self.coords.split('\n')])
    ij = np.c_[coords[:, 0], coords[:, 1]]

    if not include_k:
        return ij.astype(np.int64)

    k = np.ones(ij.shape[0]) * self.image_z_position
    return np.c_[ij, k].astype(np.int64)

Contour.to_matrix = patched_to_matrix

def calculate_malignancy(nodule):
    list_of_malignancy = [annotation.malignancy for annotation in nodule]
    malignancy = median_high(list_of_malignancy)
    return 'malignant' if malignancy > 3 else 'benign'

# Process DICOM files with the patched Contour class
for base_dir in base_dirs:
    for dirpath, _, filenames in os.walk(base_dir):
        for filename in filenames:
            if filename.endswith(".dcm"):
                try:
                    dicom_file = os.path.join(dirpath, filename)
                    dicom_data = pydicom.dcmread(dicom_file)
                    pid = dicom_data.PatientID
                    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

                    if scan is None:
                        continue

                    nodules_annotation = scan.cluster_annotations()

                    try:
                        if len(nodules_annotation) > 0:
                            contains_malignant = False

                            for nodule in nodules_annotation:
                                malignancy = calculate_malignancy(nodule)
                                if malignancy == 'malignant':
                                    contains_malignant = True
                                    break

                            if contains_malignant:
                                if malignant_count >= max_malignant_count:
                                    continue
                                malignancy = 'malignant'
                                malignant_count += 1
                            else:
                                if benign_count >= max_benign_count:
                                    continue
                                malignancy = 'benign'
                                benign_count += 1

                            img = dicom_data.pixel_array  # Assuming DICOM contains pixel data directly
                            img = cv2.resize(img, (299, 299))  
                            img = img.astype(np.float32)

                            images.append(img)
                            labels.append(malignancy)

                            print(f"Processed {malignant_count + benign_count} files")
                            print(f"Malignancy: {malignancy}")
                            print(f"Patient ID: {pid}")

                        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
                            break
                    except Exception as e:
                        # Skip problematic nodules that cannot be grouped manually
                        print(f"Skipping nodule processing for Patient ID {pid} due to error: {e}")

                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
            break

print(f"Total malignant images: {malignant_count}")
print(f"Total benign images: {benign_count}")


Processed 1 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 2 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 3 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 4 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 5 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 6 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 7 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 8 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 9 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 10 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 11 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 12 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 13 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 14 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 15 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 16 files
Malignancy: ben

In [7]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [8]:
# Normalization
images = images/255.0

In [9]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded)

# Print the mapping of labels to their encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'benign': 0, 'malignant': 1}


In [10]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42, shuffle=True)

# Convert to numpy arrays and reshape
X_train = np.array(X_train).reshape(-1, 299, 299, 1)
X_test = np.array(X_test).reshape(-1, 299, 299, 1)

# Define number of classes
num_classes = len(np.unique(labels_encoded))

print(num_classes)

2


In [11]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


# Define inception module function
def inception_module(x, f1, f2, f3):
    conv1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(x)
    conv3 = Conv2D(f2, (3, 3), padding='same', activation='relu')(x)
    conv5 = Conv2D(f3, (5, 5), padding='same', activation='relu')(x)
    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    out = concatenate([conv1, conv3, conv5, pool], axis=channel_axis)
    return out

# Define conv2d_bn function
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1)):
    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=channel_axis, scale=False)(x)
    x = Activation('relu')(x)
    return x

# Define inception and reduction blocks
def inc_block_a(x):
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=channel_axis)
    return x
def reduction_block_a(x):
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch3x3dbl, branch_pool], axis=channel_axis)
    return x



def inc_block_b(x):
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool], axis=channel_axis)
    return x

def reduction_block_b(x):
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, strides=(2, 2), padding='valid')

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis)
    return x


def inc_block_c(x):
    branch1x1 = conv2d_bn(x, 320, 1, 1)

    branch3x3 = conv2d_bn(x, 384, 1, 1)
    branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
    branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
    branch3x3 = concatenate([branch3x3_1, branch3x3_2], axis=channel_axis)

    branch3x3dbl = conv2d_bn(x, 448, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
    branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
    branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
    branch3x3dbl = concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool], axis=channel_axis)
    return x

# Build the model
img_input = Input(shape=(299, 299, 1))  # Grayscale images
channel_axis = -1 

x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')  # 149 x 149 x 32
x = conv2d_bn(x, 32, 3, 3, padding='valid')  # 147 x 147 x 32
x = conv2d_bn(x, 64, 3, 3)  # 147 x 147 x 64

x = MaxPooling2D((3, 3), strides=(2, 2))(x)  # 73 x 73 x 64
x = conv2d_bn(x, 80, 1, 1, padding='valid')  # 73 x 73 x 80
x = conv2d_bn(x, 192, 3, 3, padding='valid')  # 71 x 71 x 192
x = MaxPooling2D((3, 3), strides=(2, 2))(x)  # 35 x 35 x 192

x = inc_block_a(x)  # 35 x 35 x 256
x = inc_block_a(x)  # 35 x 35 x 256
x = inc_block_a(x)  # 35 x 35 x 256

x = reduction_block_a(x)  # 17 x 17 x 736

x = inc_block_b(x)  # 17 x 17 x 768
x = inc_block_b(x)  # 17 x 17 x 768
x = inc_block_b(x)  # 17 x 17 x 768
x = inc_block_b(x)  # 17 x 17 x 768

x = reduction_block_b(x)  # 8 x 8 x 1280

x = inc_block_c(x)  # 8 x 8 x 2048
x = inc_block_c(x)  # 8 x 8 x 2048

x = GlobalAveragePooling2D(name='avg_pool')(x)  # shape=(None, 2048)

classes = 2  
x = Dense(classes, activation='softmax', name='predictions')(x)  # shape=(None, 1000)

# Create model
model = Model(img_input, x, name='inception_v3')



In [12]:
model.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,248 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,496 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,200 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,432 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 20,032,802 (76.42 MB)

 Trainable params: 20,000,034 (76.29 MB)

 Non-trainable params: 32,768 (128.00 KB)

In [16]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization, Activation, concatenate, AveragePooling2D

# Number of folds
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Placeholder array to store performance metrics
accuracy_scores = []

# Function to create the Inception model
def create_model():
    img_input = Input(shape=(299, 299, 1))  # Grayscale images
    channel_axis = -1 

    # Initial conv and pooling layers
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')  # 149 x 149 x 32
    x = conv2d_bn(x, 32, 3, 3, padding='valid')  # 147 x 147 x 32
    x = conv2d_bn(x, 64, 3, 3)  # 147 x 147 x 64

    x = MaxPooling2D((3, 3), strides=(2, 2))(x)  # 73 x 73 x 64
    x = conv2d_bn(x, 80, 1, 1, padding='valid')  # 73 x 73 x 80
    x = conv2d_bn(x, 192, 3, 3, padding='valid')  # 71 x 71 x 192
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)  # 35 x 35 x 192

    # Inception blocks and reductions
    x = inc_block_a(x)  # 35 x 35 x 256
    x = inc_block_a(x)  # 35 x 35 x 256
    x = inc_block_a(x)  # 35 x 35 x 256
    x = reduction_block_a(x)  # 17 x 17 x 736
    x = inc_block_b(x)  # 17 x 17 x 768
    x = inc_block_b(x)  # 17 x 17 x 768
    x = inc_block_b(x)  # 17 x 17 x 768
    x = reduction_block_b(x)  # 8 x 8 x 1280
    x = inc_block_c(x)  # 8 x 8 x 2048
    x = GlobalAveragePooling2D(name='avg_pool')(x)  # shape=(None, 2048)

    # Output layer for binary classification
    output = Dense(2, activation='softmax', name='predictions')(x)  # Use sigmoid for binary classification

    # Create the model
    model = Model(img_input, output, name='InceptionV3')
    return model

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    print(f'Fold {fold + 1}/{k}')
    
    # Split data into training and validation sets for the fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Initialize a new model for each fold
    model = create_model()
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Filepath to save the best model of this fold
    checkpoint_filepath = f'best_model_fold_{fold + 1}.keras'
    
    # ModelCheckpoint to save the best weights
    checkpoint = ModelCheckpoint(checkpoint_filepath, 
                                 monitor='val_accuracy', 
                                 save_best_only=True, 
                                 mode='max', 
                                 verbose=1)

    # EarlyStopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_accuracy', 
                                   patience=4,  # Stop training if no improvement after 4 epochs
                                   mode='max',
                                   verbose=1,
                                   restore_best_weights=True)
    
    # Train the model with early stopping and checkpoint callbacks
    model.fit(X_train_fold, y_train_fold, 
              epochs=5,  # You can increase the number of epochs if needed
              batch_size=32, 
              validation_data=(X_val_fold, y_val_fold),
              verbose=1,
              callbacks=[checkpoint, early_stopping])

    # Load the best model saved during this fold
    model = load_model(checkpoint_filepath)

    # Evaluate the model on the validation set
    scores = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracy_scores.append(scores[1])  # Save validation accuracy from the evaluation

# Calculate the average accuracy across all folds
avg_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy across {k}-folds: {avg_accuracy:.4f}')


Fold 1/5
Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step - accuracy: 0.6812 - loss: 0.6675
Epoch 1: val_accuracy improved from -inf to 0.53750, saving model to best_model_fold_1.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 109s 404ms/step - accuracy: 0.6819 - loss: 0.6662 - val_accuracy: 0.5375 - val_loss: 3.0673
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - accuracy: 0.9169 - loss: 0.2240
Epoch 2: val_accuracy improved from 0.53750 to 0.67768, saving model to best_model_fold_1.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 51s 365ms/step - accuracy: 0.9171 - loss: 0.2235 - val_accuracy: 0.6777 - val_loss: 1.7333
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - accuracy: 0.9596 - loss: 0.1099
Epoch 3: val_accuracy improved from 0.67768 to 0.90714, saving model to best_model_fold_1.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 51s 367ms/step - accuracy: 0.9596 - loss: 0.1097 - val_accuracy: 0.9071 - val_loss: 0.2323
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.9869 - loss: 0.04